# AllOfUs

* **Project:** ADRD-SORL1-Biobanks
* **Version:** Python/3.10
* **Last Updated:** 14-Jun-2025

## Notebook Overview
Create cohorts (AD, Dementia, PD, Control), gene characterization, Genotools prep and application, allele freqs

# Query All of Us to create cohorts (AD, Dementia, PD, and Control), and characterize the genes of interest

## Variables used 
- `${ANCESTRY}` = EUR, AFR, AMR, AAC, AJ, MDE, SAS, CAS, EAS, FIN, CAH
- `${COHORT}` = AD, Dementia, PD, Control or Case, Control
- `${COUNT}` = Number of total individuals in each ancestry
- `${Gene}` = SORL1

## AD Cohort

In [ ]:
import pandas
import os

#This query represents dataset "Alzheimer's disease and controls" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    378419
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (378419) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    )  
                                UNION
                                DISTINCT SELECT
                                    distinct person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 65 AND 120 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (372887) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 1 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) 
                                                            AND cb_search_person.person_id NOT IN (
                                                                SELECT
                                                                    criteria.person_id 
                                                                FROM
                                                                    (SELECT
                                                                        DISTINCT person_id,
                                                                        entry_date,
                                                                        concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                    WHERE
                                                                        (
                                                                            concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740947) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740684) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1384582) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740980)
                                                                        )) criteria ) 
                                                            )
                                                        )) c_occurrence 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                        ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                        ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                        ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                        ON v.visit_concept_id = visit.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                        ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                        ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_34109098_condition_df = pandas.read_gbq(
    dataset_34109098_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_condition_df.head(5)

dataset_34109098_condition_df.to_csv("dataset_condition_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "Alzheimer's disease and controls" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1740639,1585855,1586134
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (378419) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                )  
                            UNION
                            DISTINCT SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 65 AND 120 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (372887) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) 
                                                        AND cb_search_person.person_id NOT IN (
                                                            SELECT
                                                                criteria.person_id 
                                                            FROM
                                                                (SELECT
                                                                    DISTINCT person_id,
                                                                    entry_date,
                                                                    concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                WHERE
                                                                    (
                                                                        concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740947) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740684) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1384582) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740980)
                                                                    )) criteria ) 
                                                        )
                                                    )"""

dataset_34109098_survey_df = pandas.read_gbq(
    dataset_34109098_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_survey_df.head(5)
dataset_34109098_survey_df.to_csv("dataset_survey_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "Alzheimer's disease and controls" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (378419) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        )  
                    UNION
                    DISTINCT SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740947) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740684) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1384582) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740980)
                                                            )) criteria ) 
                                                )"""

dataset_34109098_person_df = pandas.read_gbq(
    dataset_34109098_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_person_df.head(5)
dataset_34109098_person_df.to_csv("dataset_person_df.csv", index=False)

In [ ]:
person_condition_df = pd.merge(left=dataset_34109098_person_df, right=dataset_34109098_condition_df)
person_condition_df.shape

person_condition_df.to_csv("person_condition_alz_df.csv", index=False)

!grep Alzheimer person_condition_alz_df.csv | cut -f1 -d "," | sort -u | uniq > person_condition_alz_df_uniqid

## Dementia Cohort

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    4182210
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (4182210) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    )  
                                UNION
                                DISTINCT SELECT
                                    distinct person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 65 AND 120 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (372887) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 1 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) 
                                                            AND cb_search_person.person_id NOT IN (
                                                                SELECT
                                                                    criteria.person_id 
                                                                FROM
                                                                    (SELECT
                                                                        DISTINCT person_id,
                                                                        entry_date,
                                                                        concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                    WHERE
                                                                        (
                                                                            concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740947) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740684) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1384582) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740980)
                                                                        )) criteria ) 
                                                            )
                                                        )) c_occurrence 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                        ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                        ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                        ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                        ON v.visit_concept_id = visit.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                        ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                        ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_58854988_condition_df = pandas.read_gbq(
    dataset_58854988_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_condition_df.head(5)
dataset_58854988_condition_df.to_csv("dataset_conditiondem_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1586134,1585855,1740639
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (4182210) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                )  
                            UNION
                            DISTINCT SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 65 AND 120 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (372887) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) 
                                                        AND cb_search_person.person_id NOT IN (
                                                            SELECT
                                                                criteria.person_id 
                                                            FROM
                                                                (SELECT
                                                                    DISTINCT person_id,
                                                                    entry_date,
                                                                    concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                WHERE
                                                                    (
                                                                        concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740947) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740684) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1384582) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740980)
                                                                    )) criteria ) 
                                                        )
                                                    )"""

dataset_58854988_survey_df = pandas.read_gbq(
    dataset_58854988_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_survey_df.head(5)
dataset_58854988_survey_df.to_csv("dataset_surveydem_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (4182210) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        )  
                    UNION
                    DISTINCT SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740947) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740684) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1384582) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740980)
                                                            )) criteria ) 
                                                )"""

dataset_58854988_person_df = pandas.read_gbq(
    dataset_58854988_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_person_df.head(5)
dataset_58854988_person_df.to_csv("dataset_persondem_df.csv", index=False)

In [ ]:
person_condition_df = pd.merge(left=dataset_58854988_person_df, right=dataset_58854988_condition_df)
person_condition_df.shape

person_condition_df.to_csv("person_condition_dem_df.csv", index=False)

!grep dementia person_condition_dem_df.csv | cut -f1 -d "," | sort -u |  uniq > person_condition_dem_df_uniqid


## PD Cohort

In [ ]:
import pandas
import os

# This query represents dataset "PD and controls" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_65996511_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                age_at_consent BETWEEN 65 AND 120 ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (372887)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (43529272)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (376337)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (4011630)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria )  
        UNION
        DISTINCT SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (381270)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) )"""

dataset_65996511_person_df = pandas.read_gbq(
    dataset_65996511_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_65996511_person_df.head(5)
dataset_65996511_person_df.to_csv("dataset_person_PD_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "PD and controls" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_65996511_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1586134, 1585855, 1740639)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    age_at_consent BETWEEN 65 AND 120 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (372887)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (43529272)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (376337)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (4011630)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria )  
            UNION
            DISTINCT SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (381270)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) )
        )"""

dataset_65996511_survey_df = pandas.read_gbq(
    dataset_65996511_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_65996511_survey_df.head(5)
dataset_65996511_survey_df.to_csv("dataset_survey_PD_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "PD and controls" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_65996511_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (37395785, 381270)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        age_at_consent BETWEEN 65 AND 120 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (372887)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (43529272)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (376337)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (4011630)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria )  
                UNION
                DISTINCT SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (381270)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) )
            )
        ) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_65996511_condition_df = pandas.read_gbq(
    dataset_65996511_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_65996511_condition_df.head(5)
dataset_65996511_condition_df.to_csv("dataset_condition_PD_df.csv", index=False)

In [ ]:
person_condition_df = pd.merge(left=dataset_65996511_person_df, right=dataset_65996511_condition_df)
person_condition_df.shape

person_condition_df.to_csv("person_condition_PD_df.csv", index=False)

!grep Parkinson person_condition_PD_df.csv | cut -f1 -d "," | sort -u |  uniq > person_condition_PD_df_uniqid

## Control Cohort

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    378419, 4182210
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    age_at_consent BETWEEN 65 AND 120 
                            ) 
                            AND cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_whole_genome_variant = 1 
                            ) 
                            AND cb_search_person.person_id NOT IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (372887) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (43529272) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 0 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 0 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) ))
                                                ) c_occurrence 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                    ON v.visit_concept_id = visit.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_31844651_condition_df = pandas.read_gbq(
    dataset_31844651_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_condition_df.head(5)
dataset_31844651_condition_df.to_csv("controlcondition_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1586134,1585710,1740639,1585855
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (43529272) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 0 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 0 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) ))"""

dataset_31844651_survey_df = pandas.read_gbq(
    dataset_31844651_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_survey_df.head(5)
dataset_31844651_survey_df.to_csv("controlsurvey_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        age_at_consent BETWEEN 65 AND 120 
                ) 
                AND cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                ) 
                AND cb_search_person.person_id NOT IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (372887) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (43529272) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 0 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 0 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) )"""

dataset_31844651_person_df = pandas.read_gbq(
    dataset_31844651_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_person_df.head(5)
dataset_31844651_person_df.to_csv("controlperson_df.csv", index=False)

In [ ]:
!cut -f1 -d "," controlperson_df.csv | sort -u | uniq > control_ids

# Characterization of genes of interest in the Cohorts

In [ ]:
from datetime import datetime
import os
import pandas
start = datetime.now()

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
#import pyspark
import hail as hl
hl.init(default_reference = "GRCh38")

In [ ]:
mt_wgs_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH")
mt_wgs_path

In [ ]:
mt_wgs_path = "${wgs_path}/v7/wgs/short_read/snpindel/exome/splitMT/hail.mt"

In [ ]:
mt = hl.read_matrix_table(mt_wgs_path)
mt.count()

In [ ]:
test_intervals = ['chr:Start-End']

In [ ]:
mt = hl.filter_intervals(
    mt,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])
mt.count()

In [ ]:
mt.locus.summarize()

In [ ]:
flagged_samples = "${wgs_path}/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv"

In [ ]:
!gsutil -u $$GOOGLE_PROJECT cat $flagged_samples | head -n 3

In [ ]:
sample_to_remove = hl.import_table(flagged_samples, key="sample_id")

In [ ]:
mt = mt.anti_join_cols(sample_to_remove)
mt.count()

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.90, keep = True)
mt.count()

In [ ]:
out_vcf = f'{bucket}/data/${Gene}.vcf.bgz'
out_vcf

In [ ]:
hl.export_vcf(mt, out_vcf, tabix=False)

In [ ]:
!bcftools view -S person_condition_${COHORT}_df_uniqid ${Gene}.vcf.bgz --force-samples > ${Gene}_${COHORT}.vcf
!grep CHROM ${Gene}_${COHORT}.vcf > ${Gene}_in${COHORT}
!for i in `cat ${Gene}_variants` ; do grep $i ${Gene}_${COHORT}.vcf >> ${Gene}_in${COHORT} ; done
!grep chr ${Gene}_in${COHORT} |  grep -v AC=0 | cut -f1-9

# Query All of Us to prepare data for Genotools, applying Genotools, and calculating allele frequency

## Preparing Genotype data for Genotools

In [ ]:
from datetime import datetime
import os
start = datetime.now()
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
array_plink_path_msa = f'{genomic_location}/microarray/plink'
array_plink_path_msa

In [ ]:
!gsutil -u $GOOGLE_PROJECT -m cp ${wgs_path}/v7/microarray/plink/* .

In [ ]:
## Create Sample IDs file and Keep sample IDs from the all array data
nano Allsampleids_PCA   
!awk '{print "0", $1}' Allsampleids_PCA > Allsampleids_PCA_FIDIID 
!cut -f2 Allsampleids_PCA_FIDIID -d " " | sed '/^$/d' > Allsampleids_PCA_FIDIID_spaceremoved
!awk '{print "0", $1}' Allsampleids_PCA_FIDIID_spaceremoved > Allsampleids_PCA_FIDIID_spaceremoved_fam_samples
!plink --bfile arrays --keep Allsampleids_PCA_FIDIID_spaceremoved_fam_samples --make-bed --out arrays_selected_samples_keep

## Apply Genotools 

In [ ]:
!pip install the-real-genotools

!git clone https://github.com/dvitale199/GenoTools
!cd GenoTools && pip install .

!/home/jupyter/.local/bin/genotools-download
!/home/jupyter/.local/bin/genotools --bfile /path/ --out /path/ --ancestry --ref_panel /path/ --ref_labels /path/

In [ ]:
import pandas as pd
import subprocess
import sys
import numpy as np
import os
import json
import plotly.express as px
import joblib
import numba
import sklearn
import pickle as pkl
from sklearn.preprocessing import LabelEncoder

In [ ]:
pkl_path = '${WORK_DIR}/GenoTools/genotools/nba_v1.pkl'

In [ ]:
pkl_in = open(pkl_path, 'rb')
pipe_clf = pkl.load(pkl_in)
pkl_in.close()

In [ ]:
ref_pca_path = '${WORK_DIR}/arrays_selected_samples_keep_genotools_labeled_ref_pca.txt'

ref_pca = pd.read_csv(ref_pca_path, sep='\t')
print(ref_pca.head())
print(ref_pca.shape)

y = ref_pca['label']
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
proj_pca_path = '${WORK_DIR}/arrays_selected_samples_keep_genotools_projected_new_pca.txt'

proj_pca = pd.read_csv(proj_pca_path, sep='\t')
print(proj_pca.head())
print(proj_pca.shape)

In [ ]:
proj_for_pred = proj_pca.drop(columns=['FID','IID','label'], axis=1)
print(proj_for_pred.head())
print(proj_for_pred.shape)

In [ ]:
y_pred = pipe_clf.predict(proj_for_pred)

In [ ]:
y_pred = pd.Series(le.inverse_transform(y_pred))
print(y_pred.value_counts())
print(y_pred.head())

In [ ]:
proj_ids = proj_pca[['FID','IID']]
print(proj_ids.head())

In [ ]:
labels = pd.concat([proj_ids,y_pred], axis=1)
labels.columns = ['FID','IID','label']
print(labels.head())
print(labels.shape)

In [ ]:
labels.to_csv('${WORK_DIR}/arrays_selected_samples_keep_genotools_predicted_labels.txt', sep='\t', index=False)
!cat arrays_selected_samples_keep_genotools_predicted_labels.txt

In [ ]:
## Seperating AD, Dementia, PD, and Controls cohorts
!awk 'NR==FNR{a[$1]; next} $2 in a' ${COHORT}_sampleid arrays_selected_samples_keep_genotools_predicted_labels.txt > filtered_${COHORT}_sampleid_genotools.txt

In [ ]:
## Count the number of individuals in each ancestry 
!awk '$3 == "${ANCESTRY}" {count++} END {print count}' filtered_${COHORT}_sampleid_genotools.txt

## Using Genotools for calculating allele frequencies

In [ ]:
!awk '{print $2 "\t" $2 "\t" $3}' filtered_${COHORT}_sampleid_genotools.txt > filtered_${COHORT}_sampleid_genotoolsFIDIID.txt
!grep ${ANCESTRY} filtered_${COHORT}_sampleid_genotoolsFIDIID.txt> filtered_${COHORT}_sampleid_genotools_${ANCESTRY}FIDIID
!plink2 --vcf ${Gene}.vcf.bgz --make-bed --out ${Gene}_largeplink  --set-missing-var-ids @:#\$r-\$a --double-id --new-id-max-allele-len 80
!plink --merge-list merge_list_genes --make-bed --out ALL_GENES_plink
!plink2 --bfile ALL_GENES_plink  --make-bed --out ${COHORT}_${ANCESTRY}_ALLgenes --new-id-max-allele-len 80 --double-id --keep filtered_${COHORT}_sampleid_genotools_${ANCESTRY}FIDIID --set-missing-var-ids @:#\$r-\$a
!plink --bfile ${COHORT}_${ANCESTRY}_ALLgenes --extract All_variants_for_freq_genotools --freq --out FREQ_${COHORT}_${ANCESTRY}_genotools
!cat FREQ_${COHORT}_${ANCESTRY}_genotools.frq

# Query All of Us for phenotypic data

In [ ]:
## Number of Homozygous/Heterozygous
!bcftools query -i 'POS == ${}' -f "[%GT\n]" ${Gene}_${COHORT}.vcf | awk '/0\/0/ {hom_ref++} /0\/1/ {het++} /1\/1/ {hom_alt++} END {print "Homozygous Reference:", hom_ref, "\nHeterozygous:", het, "\nHomozygous Alternate:", hom_alt}'

In [ ]:
## Check data for one sample ID
!grep CHROM ${Gene}_${COHORT}.vcf > ${Gene}_${COHORT}
!grep Position ${Gene}_${COHORT}.vcf >> ${Gene}_${COHORT}
!awk -F '\t' 'NR==1 {for (i=10; i<=NF; i++) sampleIds[i-9]=$i; print "Sample_IDs"; next} {for (i=10; i<=NF; i++) {split($i, gt, ":"); if (gt[1] == "0/1") print sampleIds[i-9]}}'  ${Gene}_${COHORT}
!grep ${Sample_IDs} dataset_person_df.csv| head -30
!gsutil -u $GOOGLE_PROJECT cat "${wgs_path}/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv" | grep ${Sample_IDs}
## Check data for Sample IDs
!grep ${Sample_IDs} dataset_condition_df.csv
!grep ${Sample_IDs} dataset_survey_df.csv
!grep ${Sample_IDs} dataset_person_df.csv

In [ ]:
import pandas as pd

file_path = 'dataset_condition_pheno_df.csv'
df = pd.read_csv(file_path)

person_ids = [Sample IDs]

filtered_df = df[df['person_id'].isin(person_ids)]

result_df = filtered_df[[df.columns[14]]]

pd.set_option('display.max_rows', None)
print(result_df)

result_df.to_csv('SampleIDs_source_concept_names.csv', index=False)

In [ ]:
import pandas as pd

file_path = 'dataset_survey_pheno_df.csv'
df = pd.read_csv(file_path)

person_ids = [Sample IDs]

filtered_df = df[df['person_id'].isin(person_ids)]

result_df = filtered_df[['answer']]

pd.set_option('display.max_rows', None)
print(result_df)

result_df.to_csv('Sample IDs_answer_names.csv', index=False)